In [1]:
#Bing API を使用して画像収集を行う

In [2]:
import requests
import os
import io
import imghdr
import uuid

In [3]:
#検索
keyword = []
#欲しい枚数(50の倍数)
wantnum = 200

In [10]:
# Bing Search APIの仕様で決まっている1リクエストあたりの画像取得最大枚数
ONE_SEARCH_LIMIT = 50
# Bing Search API URL
ROOT_URL = 'https://api.cognitive.microsoft.com/bing/v5.0/images/search'
# API
API_KEY = 'API_KEY'
#Country (画像収集を行う地域を指定するMarket Codes)
MKT = 'ja-JP'
# 画像のダウンロードのタイムアウト(秒)
TIMEOUT = 5
# ダウンロードした画像を格納するディレクトリ名
SAVE_DIR = "o_picts"
if not os.path.isdir(SAVE_DIR):
    os.mkdir(SAVE_DIR)

In [11]:
def bing_search(query,offset):
    # headers
    headers = {'Ocp-Apim-Subscription-Key': API_KEY}
    #parameters: query offset country count
    payload = {'q': query, 'offset': offset, 'mkt': MKT, 'count': ONE_SEARCH_LIMIT}
    # make GET request
    result = requests.get(ROOT_URL, params=payload, headers=headers)
    # get JSON response
    return result.json()

In [ ]:
step = 1   
for key in keyword:
    pnum = 0
    for i in range(wantnum // ONE_SEARCH_LIMIT):
        #リクエスト
        response = bing_search(key, ONE_SEARCH_LIMIT * i)    
    
        for result in response.get('value', {}):
            #画像検索時のURL
            image_url = result['contentUrl']        
            #画像ダウンロード
            try:
                response_image = requests.get(image_url, timeout=TIMEOUT)
                image_binary = response_image.content
            except:
                continue

            #画像形式フィルタ (jpg or png)
            with io.BytesIO(image_binary) as fh:
                image_type = imghdr.what(fh)

                if image_type == 'jpeg':
                    extension = '.jpg'
                    pnum += 1
                elif image_type == 'png':
                    extension = '.png'
                    pnum += 1
                else:
                    continue
                #filename = str(uuid.uuid4()) + extension
                filename = str(pnum) + extension

            #画像保存
            save_dir = SAVE_DIR +"/"+'ims_' + str(step)
            if not os.path.isdir(save_dir):
                os.mkdir(save_dir)
            with open(os.path.join(save_dir, filename), 'wb') as f:
                f.write(image_binary)
    step += 1

#参考したページ
#Cognitive Services APIs
url1 = "https://www.microsoft.com/cognitive-services/en-us/"
#Market Codes
url2 = "https://msdn.microsoft.com/en-us/library/dn783426.aspx"
#parameter
url3 = "http://so-zou.jp/web-app/tech/web-api/bing/search/web-search.htm"
#画像収集
url4 = "http://techblog.adish.co.jp/entry/2016/07/07/Python%E3%81%A7Bing_Search_API%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E7%94%BB%E5%83%8F%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E9%9B%86%E3%82%81%E3%82%8B%E6%96%B9%E6%B3%95"

end